In [1]:
#load libraries
import ee
import datetime
import fiona
from shapely.geometry import shape

In [32]:
#set important variables that probably do change from time to time
extent_shapefile_path = '/home/dan/Documents/react_data/Cities_React/Boundaries.shp'
modis_product = 'MODIS/006/MOD11A2'
product = 'MOD11A2'
start_year = 2004
end_year = 2004
end_year = end_year + 1 #because how range works
variables = ['QC_Day'] #['LST_Day_1km','QC_Day']
target_crs = 'SR-ORG:6974'

In [3]:
#load the shapefile
boundaries = fiona.open(extent_shapefile_path)
the_cities = list(boundaries.items())

In [4]:
#Start earth engine
ee.Initialize()
#pull the product
imgs = ee.ImageCollection(modis_product)

In [35]:
#for each product
for year in range(start_year, end_year):
    for vari in variables:
        start = str(year)+'-'+'01-01'
        end = str(year)+'-'+'12-31'
        dat = imgs.filterDate(start, end).select(vari).toList(1000);

        #for each city
        for cit in range(1,2): #range(0,len(the_cities)):
            city_name = the_cities[cit][1]['properties']['Name']
            bbox = shape(the_cities[cit][1]['geometry']).bounds
            #print vari, city_name, bbox

            city_name = city_name.replace(' ','')



            result = dat.get(0)
            
            #create a polygon to clip things too
            # xMin, yMin, xMax, yMax
            target_projection = ee.Projection(target_crs)
            thecity = ee.Geometry.Rectangle(bbox).transform(target_projection,250)

            #convert an image collection into an image
            for i in range(0,dat.length().getInfo()):
                result = ee.Image.cat([result, dat.get(i)])

            result = result.clip(thecity)
            #request to download
            #https://stackoverflow.com/questions/39219705/how-to-download-images-using-google-earth-engines-python-api
            job_id = city_name + "_" + product +'_'+vari+'_'+str(year)

            task = ee.batch.Export.image.toDrive(image = result, folder = 'modis', 
                                                 description = job_id, fileNamePrefix = job_id)
            #print(job_id)
            #task.start()   

In [36]:
print result.getInfo().get('bands')[0]

{u'origin': [26237, 11586], u'crs': u'SR-ORG:6974', u'crs_transform': [926.625433056, 0.0, -20015109.354, 0.0, -926.625433055, 10007554.677], u'dimensions': [66, 70], u'data_type': {u'max': 255, u'type': u'PixelType', u'precision': u'int', u'min': 0}, u'id': u'QC_Day'}


In [38]:
thecity.getInfo()

{u'coordinates': [[[4297567.405786421, -792592.0231726365],
   [4351860.40840842, -792592.0231726365],
   [4357104.741394742, -728628.2763229674],
   [4302746.311447907, -728628.2763229674],
   [4297567.405786421, -792592.0231726365]]],
 u'crs': {u'properties': {u'name': u'SR-ORG:6974'}, u'type': u'name'},
 u'type': u'Polygon'}

In [39]:
#task.start()

In [40]:
bbox

(38.94993166800003, -7.127943276999929, 39.44200277300007, -6.552704129999938)

In [41]:
type(bbox)

tuple